In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 365 kB 53.5 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 6.6 MB 47.7 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 212 kB 41.8 MB/s 
     |████████████████████████████████| 141 kB 46.3 MB/s 
     |████████████████████████████████| 127 kB 60.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [ ]:
import transformers
import datasets
import numpy as np
import keras

In [ ]:
raw_dataset=datasets.load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer=transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
t=tokenizer(raw_dataset["train"][0]["tokens"],is_split_into_words=True)

In [ ]:
t.tokens()

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [ ]:
labels_name=raw_dataset["train"].features["ner_tags"].feature.names

In [ ]:
raw_dataset["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [ ]:
begin2inside={
    1:2,
    3:4,
    5:6,
    7:8
}

In [ ]:
def align_targets(labels,word_ids):
  aligned_labels=[]
  last_word=None
  for word in word_ids:
    if word is None:
      label=-100
    elif word!=last_word:
      label=labels[word]
    else:
       label=labels[word]
       if label in begin2inside:
         label=begin2inside[label]
    aligned_labels.append(label) 
    last_word=label    
  return  aligned_labels

In [ ]:
def tokenize_fn(batch):
  token_input=tokenizer(batch["tokens"],is_split_into_words=True,truncation=True)
  labels_batch=batch["ner_tags"]
  aligned_labels_batch=[]
  for i,labels in enumerate(labels_batch):
    words_ids=token_input.word_ids(i)
    aligned_labels_batch.append(align_targets(labels,words_ids))
  token_input["labels"]=aligned_labels_batch
  return token_input

In [ ]:
tokenized_dataset=raw_dataset.map(tokenize_fn,batched=True,remove_columns=raw_dataset["train"].column_names)

  0%|          | 0/15 [00:00<?, ?ba/s]

In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=0a1773f14391d3ff715db61b989464c34eb2ab6cdf0c56a44b524e68460e32ed
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
metrics=datasets.load_metric("seqeval")

In [ ]:
def compute_metrics(logits_and_labels):
  logits,labels=logits_and_labels
  preds=np.argmax(logits,axis=-1)
  str_label=[[labels_name[t] for t in label if t!=-100] for label in labels]
  str_pred=[[labels_name[p] for p,t in zip(pred,target) if t!=-100]for pred,target in zip(preds,labels)]
  the_metrics=metrics.compute(predications=str_pred,references=str_label)
  return {
      "precision":the_metrics["overall_precision"],
      "recall":the_metrics["overall_recall"],
      "f1":the_metrics["overall_f1"],
      "accuracy":the_metrics["overall_accuracy"]
  }

In [ ]:
id2label={k:v for k,v in enumerate(labels_name)}
label2id={v:k for k,v in id2label.items()}

In [ ]:
model=transformers.AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased",id2label=id2label,label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

In [ ]:
training_args=transformers.TrainingArguments(
    "distilbert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=2e-5
)

In [ ]:
data_collector=transformers.DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
train=transformers.Trainer(model,
                           args=training_args,
                           train_dataset=tokenized_dataset["train"],
                           eval_dataset=tokenized_dataset["validation"],
                           tokenizer=tokenizer,
                           data_collator=data_collector,
                           compute_metrics=compute_metrics)

In [ ]:
train.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5268


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8


TypeError: ignored